In [1]:
import sys, os, time
import numpy as np
import pandas as pd
import pickle

from sklearn.preprocessing import label_binarize

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from estimators.statistical_descriptor import Nagler_WS
# from plot.figure_roc import ROC_plot
from utils.dataset_management import parse_pipeline
from utils.dataset_load import shuffle_data, DatasetLoader
from utils.fold_management import FoldManagement
from utils.label_management import LabelManagement
from utils.figures import plot_boxplots, plot_roc_curves
from utils.files_management import (
    load_yaml,
    dump_pkl,
    init_logger,
    report_prediction,
    report_metric_from_log,
    set_folder,
    logger_dataset,
    logger_fold
)

In [2]:
os.cpu_count()

8

In [20]:
with open('../results/study_120624_09H39M55_crocus_combinationFold/models/KNN_direct/metrics.pkl', 'rb') as f:
    data = pickle.load(f)
data

[{'f1_score_macro': 87.48,
  'f1_score_weighted': 87.63,
  'accuracy_score': 87.6,
  'precision_score_macro': 87.35000000000001,
  'recall_score_macro': 87.74,
  'roc_auc_score': 87.74,
  'log_loss': 32.33,
  'kappa_score': 74.98,
  'confusion_matrix':          0        1
  0  86.6397  13.3603
  1  11.1688  88.8312,
  'y_true': array([0, 0, 0, ..., 0, 1, 0]),
  'y_pred': array([[0.88, 0.12],
         [0.98, 0.02],
         [0.94, 0.06],
         ...,
         [0.92, 0.08],
         [0.2 , 0.8 ],
         [0.98, 0.02]]),
  'fold': 0,
  'training_time': 10.756357908248901,
  'prediction_time': 3.3500258922576904},
 {'f1_score_macro': 82.89999999999999,
  'f1_score_weighted': 83.06,
  'accuracy_score': 83.00999999999999,
  'precision_score_macro': 82.82000000000001,
  'recall_score_macro': 83.25,
  'roc_auc_score': 83.25,
  'log_loss': 43.64,
  'kappa_score': 65.85,
  'confusion_matrix':          0        1
  0  81.2441  18.7559
  1  14.7482  85.2518,
  'y_true': array([0, 1, 1, ..., 0, 1

In [21]:
for metrics in data:
    metrics_tmp = metrics
    print(f"__________________ Fold {metrics_tmp["fold"]} with {"name"} __________________")
    metrics_tmp.pop("fold")
    metrics_tmp.pop("y_pred")
    metrics_tmp.pop("y_true")
    for metric, value in metrics_tmp.items():
        
        if isinstance(value, float):
            value_str = f"{value:.2f}"
        elif isinstance(value, np.ndarray):
            value_str = "\n" + str(value)
        else:
            value_str = str(value)
        print(f" {metric} : {value_str}")

__________________ Fold 0 with name __________________


TypeError: unhashable type: 'list'